In [1]:
from pprint import pprint

# I - Data preprocessing

## I.1 - Download dataset

In [2]:
# ! pip install datasets

In [3]:
from datasets import load_dataset

In [4]:
data_name = 'ncduy/mt-en-vi'
data = load_dataset(data_name)

README.md:   0%|          | 0.00/3.43k [00:00<?, ?B/s]

train.csv:   0%|          | 0.00/597M [00:00<?, ?B/s]

valid.csv:   0%|          | 0.00/2.45M [00:00<?, ?B/s]

test.csv:   0%|          | 0.00/2.43M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2884451 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11316 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11225 [00:00<?, ? examples/s]

In [5]:
sample_size = 100000
train_data = data['train'].shuffle(seed=42).select(range(sample_size))
validation_data = data['validation'].shuffle(seed=42).select(range(int(0.1*sample_size)))
test_data = data['test'].shuffle(seed=42).select(range(int(0.1*sample_size)))

In [6]:
print(train_data.shape)
pprint(train_data[0])

(100000, 3)
{'en': '"education and leading me to providing service to the Republic."',
 'source': 'OpenSubtitles v2018',
 'vi': '"nơi giáo dục và dìu dắt tôi để tôi có thể phụng sự nền Cộng Hòa"'}


## I.2 - Process data using SentencePiece

In [7]:
import sentencepiece as spm
import os

In [8]:
# Save data into files
def save_to_file(data, filename):
    with open(filename+'.en', 'w', encoding='utf-8') as f_en, open(filename+'.vi', 'w', encoding='utf-8') as f_vi:
        for sample in data:
            f_en.write(sample['en'] + '\n')
            f_vi.write(sample['vi'] + '\n')

In [9]:
save_to_file(train_data, '/kaggle/working/train')
save_to_file(validation_data, '/kaggle/working/validation')
save_to_file(test_data, '/kaggle/working/test')

In [10]:
# Training SentencePiece
def training_sentencepiece(trainfile, prefix, vocab_size=32000, type='bpe'):
    spm.SentencePieceTrainer.Train(
        input=trainfile+'.en',
        model_prefix=prefix+'_en',
        vocab_size=vocab_size,
        model_type=type
    )
    spm.SentencePieceTrainer.Train(
        input=trainfile+'.vi',
        model_prefix=prefix+'_vi',
        vocab_size=vocab_size,
        model_type=type
    )
    
    # Load trained tokenizer
    sp_en = spm.SentencePieceProcessor(model_file=prefix+"_en.model")
    sp_vi = spm.SentencePieceProcessor(model_file=prefix+"_vi.model")
    
    return sp_en, sp_vi

In [11]:
trainfile = '/kaggle/working/train'
prefix = '/kaggle/working/spm'
sp_en, sp_vi = training_sentencepiece(trainfile, prefix)

In [12]:
example_en = "education and leading me to providing service to the Republic."
example_vi = "Tôi yêu đại bàng!"

print(sp_en.encode(example_en, out_type=str))
print(sp_vi.encode(example_vi, out_type=str))

['▁education', '▁and', '▁leading', '▁me', '▁to', '▁providing', '▁service', '▁to', '▁the', '▁Republic', '.']
['▁Tôi', '▁yêu', '▁đại', '▁bàng', '!']


## I.3 - Convert data to Tensor

In [13]:
import torch
from torch.utils.data import Dataset, DataLoader

In [14]:
class TranslationDataset(Dataset):
    def __init__(self, src_texts, tgt_texts, src_tokenizer, tgt_tokenizer, max_length=128):
        self.src_texts = src_texts
        self.tgt_texts = tgt_texts
        self.src_tokenizer = src_tokenizer
        self.tgt_tokenizer = tgt_tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.src_texts)

    def __getitem__(self, idx):
        # Tokenize input (source - English)
        src_ids = self.src_tokenizer.encode(self.src_texts[idx])[:self.max_length - 1]  
        src_ids.append(self.src_tokenizer.eos_id())  # Thêm <EOS>

        # Tokennize output (target - Vietnamese)
        tgt_ids = self.tgt_tokenizer.encode(self.tgt_texts[idx])[:self.max_length - 2]  
        tgt_ids = [self.tgt_tokenizer.bos_id()] + tgt_ids + [self.tgt_tokenizer.eos_id()]  # Add <SOS> and <EOS>

        return torch.tensor(src_ids), torch.tensor(tgt_ids)


# Padding batch
def collate_fn(batch):
    src_batch, tgt_batch = zip(*batch)
    
    src_batch = torch.nn.utils.rnn.pad_sequence(src_batch, batch_first=True, padding_value=0)
    tgt_batch = torch.nn.utils.rnn.pad_sequence(tgt_batch, batch_first=True, padding_value=0)

    return {"src": src_batch, "tgt": tgt_batch}

In [15]:
train_src_texts = train_data['en'][:5000]
train_tgt_texts = train_data['vi'][:5000]

train_dataset = TranslationDataset(train_src_texts, train_tgt_texts, sp_en, sp_vi)

In [16]:
val_src_texts = validation_data['en'][:100]
val_tgt_texts = validation_data['vi'][:100]

val_dataset = TranslationDataset(val_src_texts, val_tgt_texts, sp_en, sp_vi)

In [17]:
# DataLoader
batch_size = 32
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)

In [18]:
batch = next(iter(train_dataloader))
print(batch['src'].shape)
pprint(batch['src'])

torch.Size([32, 72])
tensor([[  488, 31860, 31842,  ...,     0,     0,     0],
        [20090, 31855, 12014,  ...,     0,     0,     0],
        [10104, 31855,     5,  ...,     0,     0,     0],
        ...,
        [ 5540,   431,     8,  ...,     0,     0,     0],
        [  407,    85,     5,  ...,     0,     0,     0],
        [26688,  3192,  1487,  ...,     0,     0,     0]])


# II - Build model

In [19]:
import torch.nn as nn

In [20]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hidden_dim, num_layers, dropout=0.3, pad_idx=0):
        super().__init__()
        self.embedding = nn.Embedding(input_dim, emb_dim, padding_idx=pad_idx)
        self.rnn = nn.GRU(emb_dim, hidden_dim, num_layers, dropout=dropout, batch_first=True)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        src
        embedded = self.embedding(src)
        embedded = self.dropout(embedded)
        output, hidden = self.rnn(embedded)
        return output, hidden
        

In [21]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hidden_dim, num_layers, dropout, pad_idx=0):
        super().__init__()
        self.output_dim = output_dim
        self.embedding = nn.Embedding(output_dim, emb_dim, padding_idx=pad_idx)
        self.rnn = nn.GRU(emb_dim, hidden_dim, num_layers, dropout=dropout, batch_first=True)
        self.fc_out = nn.Linear(hidden_dim, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, input, hidden):
        input = input.unsqueeze(1)  # [batch] -> [batch, 1]
        embedded = self.embedding(input)  # [batch, 1, emb_dim]
        embedded = self.dropout(embedded)  # [batch, 1, emb_dim]
        output, hidden = self.rnn(embedded, hidden)  # output: [batch, 1, hidden_dim]
        prediction = self.fc_out(output.squeeze(1))  # [batch, output_dim]
        return prediction, hidden


In [22]:
import random

class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    def forward(self, src, trg, teacher_forcing_ratio=0.5):
        batch_size = src.shape[0]
        trg_len = trg.shape[1]
        trg_vocab_size = self.decoder.output_dim
        
        # outputs = torch.zeros(trg_len, batch_size, trg_vocab_size)
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)

        # Encode input
        output, hidden = self.encoder(src)

        # First token <SOS>
        input = trg[:, 0]

        for t in range(1, trg_len):
            output, hidden = self.decoder(input, hidden)
            outputs[t] = output

            # Teacher forcing
            teacher_force = random.random() < teacher_forcing_ratio
            top1 = output.argmax(1)

            input = trg[:, t] if teacher_force else top1

        return outputs


In [23]:
# Hyperparams
INPUT_DIM = len(sp_en)
OUTPUT_DIM = len(sp_vi)
EMB_DIM = 256
HIDDEN_DIM = 512
NUM_LAYERS = 2
DROPOUT = 0.3
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Khởi tạo Encoder & Decoder
encoder = Encoder(INPUT_DIM, EMB_DIM, HIDDEN_DIM, NUM_LAYERS, DROPOUT)
decoder = Decoder(OUTPUT_DIM, EMB_DIM, HIDDEN_DIM, NUM_LAYERS, DROPOUT)

# Khởi tạo Seq2Seq
model = Seq2Seq(encoder, decoder, DEVICE)


# III - Train and Evaluate model

In [24]:
import torch.optim as optim
import torch.nn as nn
from tqdm import tqdm


LEARNING_RATE = 0.001
PAD_IDX = 0

criterion = torch.nn.CrossEntropyLoss(ignore_index=PAD_IDX)  
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)


In [25]:
import random

def train(model, iterator, optimizer, criterion, clip, teacher_forcing_ratio=0.5):
    model.train()
    epoch_loss = 0
    model.to(DEVICE)
    for batch in tqdm(iterator, desc=f'Training: '):
        src, trg = batch['src'], batch['tgt']
        src, trg = src.to(DEVICE), trg.to(DEVICE)
        
        optimizer.zero_grad()

        output = model(src, trg, teacher_forcing_ratio)

        output_dim = output.shape[-1]
        output = output[1:].reshape(-1, output_dim)  # [batch_size * trg_len, vocab_size]
        trg = trg[:, 1:].reshape(-1)  # [batch_size * trg_len]

        loss = criterion(output, trg)
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

        optimizer.step()
        epoch_loss += loss.item()

    return epoch_loss / len(iterator)


In [29]:
def evaluate(model, iterator, criterion):
    model.eval()
    epoch_loss = 0

    with torch.no_grad():
        for src, trg in tqdm(iterator, desc='Evaluating: '):
            src, trg = batch['src'], batch['tgt']
            src, trg = src.to(DEVICE), trg.to(DEVICE)
            output = model(src, trg, teacher_forcing_ratio=0)

            output_dim = output.shape[-1]
            output = output[1:].reshape(-1, output_dim)
            trg = trg[:, 1:].reshape(-1)

            loss = criterion(output, trg)
            epoch_loss += loss.item()
    print()
    return epoch_loss / len(iterator)


In [30]:
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"


In [31]:
N_EPOCHS = 10
CLIP = 5  # Max gradient norm

train_losses = []
val_losses = []

for epoch in range(N_EPOCHS):
    train_loss = train(model, train_dataloader, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, val_dataloader, criterion)

    print(f'Epoch {epoch+1}:')
    print(f'  Train Loss: {train_loss:.3f}')
    print(f'  Valid Loss: {valid_loss:.3f}')

    train_losses.append(train_loss)
    val_losses.append(valid_loss)

Evaluating: 100%|██████████| 4/4 [00:01<00:00,  2.96it/s]



Epoch 1:
  Train Loss: 7.232
  Valid Loss: 7.471


Evaluating: 100%|██████████| 4/4 [00:01<00:00,  2.92it/s]



Epoch 2:
  Train Loss: 7.186
  Valid Loss: 7.475


Evaluating: 100%|██████████| 4/4 [00:01<00:00,  2.97it/s]


...


Epoch 8:
  Train Loss: 7.069
  Valid Loss: 7.372


Evaluating: 100%|██████████| 4/4 [00:01<00:00,  2.98it/s]



Epoch 9:
  Train Loss: 7.071
  Valid Loss: 7.341


Evaluating: 100%|██████████| 4/4 [00:01<00:00,  2.96it/s]


Epoch 10:
  Train Loss: 7.064
  Valid Loss: 7.380


# IV - Testing

In [35]:
torch.save(model.state_dict(), '/kaggle/working/seq2seq.pth')

In [123]:
model.load_state_dict(torch.load("/kaggle/working/seq2seq.pth", 
                                 map_location=torch.device("cpu"),
                                 weights_only=True))
model.eval()  

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(32000, 256, padding_idx=0)
    (rnn): GRU(256, 512, num_layers=2, batch_first=True, dropout=0.3)
    (dropout): Dropout(p=0.3, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(32000, 256, padding_idx=0)
    (rnn): GRU(256, 512, num_layers=2, batch_first=True, dropout=0.3)
    (fc_out): Linear(in_features=512, out_features=32000, bias=True)
    (dropout): Dropout(p=0.3, inplace=False)
  )
)

In [156]:
import torch

def translate_sentence(sentence, sp_en, trg_vocab, model, device, max_len=128):
    model.to(device).eval()
    
    # Tokenize input using SentencePiece
    tokens = sp_en.encode(sentence, out_type=str)  # Tokenize
    print("Tokenized input:", tokens)
    
    # Chuyển token thành chỉ mục (ID)
    src_indexes = sp_en.encode(sentence)  # SentencePiece auto convert to ID
    src_tensor = torch.tensor(src_indexes, dtype=torch.long).unsqueeze(0).to(device)  # [1, seq_len]
    with torch.inference_mode():
        _, hidden = model.encoder(src_tensor)  # Encode input
    
    trg_indexes = [trg_vocab['stoi']["<s>"]]  # Begin with <sos>
    for _ in range(max_len):
        trg_tensor = torch.tensor([trg_indexes[-1]], dtype=torch.long).to(device)  # [1]
        with torch.inference_mode():
            output, hidden = model.decoder(trg_tensor, hidden)  # Get output from decoder
        pred_token = output.argmax(1).item()  # Get token has highest probs
        trg_indexes.append(pred_token)

        if pred_token == trg_vocab['stoi']["</s>"]:  # End if <eos>
            break

    # Convert ID to word
    trg_tokens = [trg_vocab['itos'][i] for i in trg_indexes]
    return " ".join(trg_tokens[1:-1])  # Remove <sos> and <eos>


In [157]:
trg_vocab = {
    "stoi": {sp_vi.id_to_piece(i): i for i in range(sp_vi.get_piece_size())},
    "itos": {i: sp_vi.id_to_piece(i) for i in range(sp_vi.get_piece_size())},
}

sentence = "How are you?"
translated_sentence = translate_sentence(sentence, sp_en, trg_vocab, model, device="cpu")
print("Translated:", translated_sentence)


Tokenized input: ['▁How', '▁are', '▁you', '?']
Translated: .
